In [0]:
import tensorflow as f


def calc_loss_acc(labels, logits):
    """Function to compute loss value. Here, we used cross entropy 
    Args:
        logits: 4D tensor. output tensor from segnet model, which is the output of softmax
        labels: true labels tensor
    Returns:
        loss (cross_entropy_mean), accuracy, predicts(logits with softmax) 
    """
    # calc cross entropy mean  cross_entropy
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits, name='cross_entropy')
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    tf.summary.scalar(name='loss', tensor=cross_entropy_mean)


    predicts = tf.equal(tf.argmax(logits, axis=1), tf.argmax(labels, axis=1))

    accuracy = tf.reduce_mean(tf.cast(predicts, dtype=tf.float32))
    tf.summary.scalar(name='accuracy', tensor=accuracy)

    return cross_entropy_mean, accuracy, predicts


def train_op(total_loss, global_steps, base_learning_rate, option='Adam'):
    """This function defines train optimizer 
    Args:
        total_loss: the loss value
        global_steps: global steps is used to track how many batch had been passed. In the training process, the initial value for global_steps = 0, here  
        global_steps=tf.Variable(0, trainable=False). then after one batch of images passed, the loss is passed into the optimizer to update the weight, then the global 
        step increased by one.
        base_learning_rate: default value 0.1
    Returns:
        the train optimizer
    """

    # base_learning_rate = 0.01
    # get update operation
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):

        if option == 'Adam':
            optimizer = tf.train.AdamOptimizer(learning_rate=base_learning_rate)
            print("Running with Adam Optimizer with learning rate:", base_learning_rate)
        elif option == 'SGD':
            # base_learning_rate = 0.01
            learning_rate_decay = tf.train.exponential_decay(base_learning_rate, global_steps, 1000, 0.0005)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate_decay)
            print("Running with SGD Optimizer with learning rate:", learning_rate_decay)
        else:
            raise ValueError('Optimizer is not recognized')

        grads = optimizer.compute_gradients(total_loss, var_list=tf.trainable_variables())
        training_op = optimizer.apply_gradients(grads, global_step=global_steps)

    return training_op



In [0]:
import tensorflow as tf


def max_pooling(inputs, ksize, stride, padding, name):
    """Create max pooling layer 
    Args:
        inputs: float32 4D tensor
        ksize: a tuple of 2 int with (kernel_height, kernel_width)
        stride: a tuple
        padding: string. padding mode 'SAME', '
        name: string
        
    Returns:
        4D tensor of [batch_size, height, width, channels]
    """
    with tf.variable_scope(name) as scope:
        value = tf.nn.max_pool(inputs, ksize=[1, ksize[0], ksize[1], 1], strides=[1, stride[0], stride[1], 1], padding=padding, name=scope.name)
    return value

def dropout(inputs, keep_prob, name):
    """Dropout layer
    Args:
        inputs: float32 4D tensor 
        keep_prob: the probability of keep training sample
        name: layer name to define
    Returns:
        4D tensor of [batch_size, height, width, channels]
    """
    
    return tf.nn.dropout(inputs, rate=(1-keep_prob), name=name)


def norm(inputs, radius=4, name=None):
    """
    
    """
    with tf.variable_scope(name) as scope:
        value = tf.nn.lrn(inputs, depth_radius=radius, bias=1.0, alpha=1e-4, beta=0.75, name=name)
    return value

# def batch_norm(inputs, name):
#     """batch normalization layer


#     """

def conv2d(inputs, shape, padding, name):
    """Create convolution 2D layer
    Args:
        inputs: float32. 4D tensor
        shape: the shape of kernel 
        padding: string. padding mode 'SAME',
        name: corressponding layer's name
    Returns:
        Output 4D tensor
    """
    with tf.variable_scope(name) as scope:
        # get weights value and record a summary protocol buffer with a histogram
        weights = tf.get_variable('weights', shape=shape, initializer=tf.initializers.he_normal())
        tf.summary.histogram(scope.name + 'weights', weights)

        # get biases value and record a summary protocol buffer with a histogram
        biases = tf.get_variable('biases', shape=shape[3], initializer=tf.initializers.random_normal())
        tf.summary.histogram(scope.name + 'biases', biases)
        # compute convlotion W * X + b, activiation function relu function
        outputs = tf.nn.conv2d(inputs, weights, strides=[1, 1, 1, 1], padding=padding)
        outputs = tf.nn.bias_add(outputs, biases)
        outputs = tf.nn.relu(outputs, name=scope.name + 'relu')
    return outputs

def fc(inputs, shape, name):
    """Create fully collection layer 
    Args:
        inputs: Float32. 2D tensor with shape [batch, input_units]
        shape: Int. a tuple with [num_inputs, num_outputs]
        name: sring. layer name
    Returns:
        Outputs fully collection tensor
    """

    with tf.variable_scope(name) as scope:
        weights = tf.get_variable('weights', shape = [shape[0], shape[1]], initializer=tf.initializers.he_normal())
        biases = tf.get_variable('biases', shape = [shape[1]], initializer=tf.initializers.random_normal())
        # outputs = tf.nn.xw_plus_b(inputs, weights, biases, name = scope.name)
        outputs = tf.add(tf.matmul(inputs, weights), biases, name=scope.name)

    return tf.nn.relu(outputs)

In [0]:
import tensorflow as tf

def alexnet(inputs, num_classes, keep_prob):
    """Create alexnet model
    """
    x = tf.reshape(inputs, shape=[-1, 28, 28, 1])

    # first conv layer, downsampling layer, and normalization layer
    conv1 = conv2d(x, shape=(11, 11, 1, 96), padding='SAME', name='conv1')
    pool1 = max_pooling(conv1, ksize=(2, 2), stride=(2, 2), padding='SAME', name='pool1')
    norm1 = norm(pool1, radius=4, name='norm1')

    # second conv layer
    conv2 = conv2d(norm1, shape=(5, 5, 96, 256), padding='SAME', name='conv2')
    pool2 = max_pooling(conv2, ksize=(2, 2), stride=(2, 2), padding='SAME', name='pool2')
    norm2 = norm(pool2, radius=4, name='norm2')

    # 3rd conv layer
    conv3 = conv2d(norm2, shape=(3, 3, 256, 384), padding='SAME', name='conv3')
    # pool3 = max_pooling(conv3, ksize=(2, 2), stride=(2, 2), padding='SAME', name='pool3')
    norm3 = norm(conv3, radius=4, name='norm3')

    # 4th conv layer
    conv4 = conv2d(norm3, shape=(3, 3, 384, 384), padding='SAME', name='conv4')

    # 5th conv layer
    conv5 = conv2d(conv4, shape=(3, 3, 384, 256), padding='SAME', name='conv5')
    pool5 = max_pooling(conv5, ksize=(2, 2), stride=(2, 2), padding='SAME', name='pool5')
    norm5 = norm(pool5, radius=4, name='norm5')

    # first fully connected layer
    fc1 = tf.reshape(norm5, shape=(-1, 4*4*256))
    fc1 = fc(fc1, shape=(4*4*256, 4096), name='fc1')
    fc1 = dropout(fc1, keep_prob=keep_prob, name='dropout1')

    fc2 = fc(fc1, shape=(4096, 4096), name='fc2')
    fc2 = dropout(fc2, keep_prob=keep_prob, name='dropout2')

    # output logits value
    with tf.variable_scope('classifier') as scope:
        weights = tf.get_variable('weights', shape=[4096, num_classes], initializer=tf.initializers.he_normal())
        biases = tf.get_variable('biases', shape=[num_classes], initializer=tf.initializers.random_normal())
        # define output logits value
        logits = tf.add(tf.matmul(fc2, weights), biases, name=scope.name + '_logits')

    return logits



In [0]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)


flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_integer('valid_steps', 11, 'The number of validation steps ')

flags.DEFINE_integer('max_steps', 1001, 'The number of maximum steps for traing')

flags.DEFINE_integer('batch_size', 128, 'The number of images in each batch during training')

flags.DEFINE_float('base_learning_rate', 0.0001, "base learning rate for optimizer")

flags.DEFINE_integer('input_shape', 784, 'The inputs tensor shape')

flags.DEFINE_integer('num_classes', 10, 'The number of label classes')

flags.DEFINE_string('save_dir', './outputs', 'The path to saved checkpoints')

flags.DEFINE_float('keep_prob', 0.75, "the probability of keeping neuron unit")

flags.DEFINE_string('tb_path', './tb_logs/', 'The path points to tensorboard logs ')


FLAGS = flags.FLAGS



def train(FLAGS):
    """Training model
    """
    valid_steps = FLAGS.valid_steps
    max_steps = FLAGS.max_steps
    batch_size = FLAGS.batch_size
    base_learning_rate = FLAGS.base_learning_rate
    input_shape = FLAGS.input_shape  # image shape = 28 * 28
    num_classes = FLAGS.num_classes
    keep_prob = FLAGS.keep_prob
    save_dir = FLAGS.save_dir
    tb_path = FLAGS.tb_path 

    train_loss, train_acc = [], []
    valid_loss, valid_acc = [], []

    tf.reset_default_graph()
    # define default tensor graphe 
    with tf.Graph().as_default():
        images_pl = tf.placeholder(tf.float32, shape=[None, input_shape])
        labels_pl = tf.placeholder(tf.float32, shape=[None, num_classes])

        # define a variable global_steps
        global_steps = tf.Variable(0, trainable=False)

        # build a graph that calculate the logits prediction from model
        logits = alexnet(images_pl, num_classes, keep_prob)

        loss, acc, _ = calc_loss_acc(labels_pl, logits)

        # build a graph that trains the model with one batch of example and updates the model params 
        training_op = train_op(loss, global_steps, base_learning_rate)

        # define the model saver
        saver = tf.train.Saver(tf.global_variables())
        
        # define a summary operation 
        summary_op = tf.summary.merge_all()

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            # print(sess.run(tf.trainable_variables()))
            # start queue runners
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            train_writter = tf.summary.FileWriter(tb_path, sess.graph)

            # start training
            for step in range(100):
                # get train image / label batch
                train_image_batch, train_label_batch = mnist.train.next_batch(batch_size)

                train_feed_dict = {images_pl: train_image_batch, labels_pl: train_label_batch}

                _, _loss, _acc, _summary_op = sess.run([training_op, loss, acc, summary_op], feed_dict = train_feed_dict)

                # store loss and accuracy value
                train_loss.append(_loss)
                train_acc.append(_acc)
                print("Iteration " + str(step) + ", Mini-batch Loss= " + "{:.6f}".format(_loss) + ", Training Accuracy= " + "{:.5f}".format(_acc))

                if step % 100 == 0:
                    _valid_loss, _valid_acc = [], []
                    print('Start validation process')

                    for step in range(valid_steps):
                        valid_image_batch, valid_label_batch = mnist.test.next_batch(batch_size)

                        valid_feed_dict = {images_pl: valid_image_batch, labels_pl: valid_label_batch}

                        _loss, _acc = sess.run([loss, acc], feed_dict = valid_feed_dict)

                        _valid_loss.append(_loss)
                        _valid_acc.append(_acc)

                    valid_loss.append(np.mean(_valid_loss))
                    valid_acc.append(np.mean(_valid_acc))

                    print("Iteration {}: Train Loss {:6.3f}, Train Acc {:6.3f}, Val Loss {:6.3f}, Val Acc {:6.3f}".format(step, train_loss[-1], train_acc[-1], valid_loss[-1], valid_acc[-1]))
            
            np.save(os.path.join(save_dir, 'accuracy_loss', 'train_loss'), train_loss)
            np.save(os.path.join(save_dir, 'accuracy_loss', 'train_acc'), train_acc)
            np.save(os.path.join(save_dir, 'accuracy_loss', 'valid_loss'), valid_loss)
            np.save(os.path.join(save_dir, 'accuracy_loss', 'valid_acc'), valid_acc)
            checkpoint_path = os.path.join(save_dir, 'model', 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

            coord.request_stop()
            coord.join(threads)
    
if __name__ == '__main__':
    train(FLAGS)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Running with Adam Optimizer with learning rate: 0.0001
Iteration 0, Mini-batch Loss= 7.779362, Training Accuracy= 0.09375
Start validation process
Iteration 10: Train Loss  7.779, Train Acc  0.094, Val Loss  6.824, Val Acc  0.104
Iteration 1, Mini-batch Loss= 8.402212, Training Accuracy= 0.04688
Iteration 2, Mini-batch Loss= 9.462041, Training Accuracy= 0.13281
Iteration 3, Mini-batch Loss= 9.505525, Training Accuracy= 0.11719
Iteration 4, Mini-batch Loss= 8.242508, Training Accuracy= 0.10938
Iteration 5, Mini-batch Loss= 5.228747, Training Accuracy= 0.14062
Iteration 6, Mini-batch Loss= 5.201808, Training Accuracy= 0.08594
Iteration 7, Mini-batch Loss= 4.166608, Training Accuracy= 0.12500
Iteration 8, Mini-batch Loss= 3.516380, Training Accuracy= 0.15625
Iteration 9, Mini-batch Loss= 3.624369, Trai

FileNotFoundError: ignored